In [19]:
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import google.auth

def authenticate_docs_service():
    # Authenticate and build the Google Docs service.
    auth.authenticate_user()
    creds, _ = google.auth.default()
    return build("docs", "v1", credentials=creds)

def build_requests_and_text(md_content):
    """
    Parse markdown content and build a combined text string plus formatting requests.
    Returns the full text to insert and a list of formatting requests.
    """
    full_text = ""
    requests = []
    current_index = 1  # The doc always starts at index 1

    # Split content into lines.
    lines = md_content.splitlines()

    # Process each line.
    # We'll record checklist ranges in a list so we can later apply checklist formatting.
    checklist_ranges = []  # list of (start_idx, end_idx) for checklist items

    for line in lines:
        is_checkbox = False  # Initialize for each line
        original_line = line  # Keep for footer detection.
        line = line.rstrip("\n")
        style = None
        indent_level = 0

        # Determine what the line represents.
        if not line.strip():
            new_line = "\n"
        elif line.startswith("# "):
            new_line = line[2:].strip() + "\n"
            style = "HEADING_1"
        elif line.startswith("## "):
            new_line = line[3:].strip() + "\n"
            style = "HEADING_2"
        elif line.startswith("### "):
            new_line = line[4:].strip() + "\n"
            style = "HEADING_3"
        elif re.match(r"^- \[ \] ", line):
            # Markdown checkbox: Remove the '- [ ]' marker and flag as checklist item.
            new_line = line[6:].strip() + "\n"
            is_checkbox = True
        elif re.match(r"^-", line):
            # Bullet point. Count leading spaces to determine indent (assume 2 spaces per level).
            match = re.match(r"^(\s*)-\s*(.*)", line)
            if match:
                spaces = match.group(1)
                indent_level = len(spaces) // 2  # Each 2 spaces equals one indent level.
                new_line = "• " + match.group(2).strip() + "\n"
            else:
                new_line = line + "\n"
        else:
            new_line = line + "\n"

        # Record where this line starts.
        start_idx = current_index
        full_text += new_line
        current_index += len(new_line)
        end_idx = current_index

        # If this line was a checkbox, record its range for later checklist formatting.
        if is_checkbox:
            checklist_ranges.append((start_idx, end_idx))

        # If we have a heading style, add a paragraph style update.
        if style:
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": start_idx, "endIndex": end_idx},
                    "paragraphStyle": {"namedStyleType": style},
                    "fields": "namedStyleType"
                }
            })

        # If this is a bullet point with indent, update indentation.
        if indent_level > 0:
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": start_idx, "endIndex": end_idx},
                    "paragraphStyle": {"indentStart": {"magnitude": 36 * indent_level, "unit": "PT"}},
                    "fields": "indentStart"
                }
            })

        # Bold any assignee mentions (e.g., @sarah) within the line.
        for m in re.finditer(r"@\w+", new_line):
            m_start = start_idx + m.start()
            m_end = start_idx + m.end()
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": m_start, "endIndex": m_end},
                    "textStyle": {"bold": True},
                    "fields": "bold"
                }
            })

        # Style footer information (e.g., "Meeting recorded by:" or "Duration:")
        if re.search(r"^(Meeting recorded by:|Duration:)", original_line.strip()):
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": start_idx, "endIndex": end_idx},
                    "textStyle": {
                        "italic": True,
                        "foregroundColor": {"color": {"rgbColor": {"red": 0.5, "green": 0.5, "blue": 0.5}}}
                    },
                    "fields": "italic,foregroundColor"
                }
            })

    # For each recorded checklist range, add a request to convert that paragraph into a checklist.
    for start_idx, end_idx in checklist_ranges:
        requests.append({
            "createParagraphBullets": {
                "range": {"startIndex": start_idx, "endIndex": end_idx},
                "bulletPreset": "BULLET_CHECKBOX"
            }
        })

    return full_text, requests

def create_google_doc(service, title, md_content):
    try:
        # Create a new Google Doc.
        doc = service.documents().create(body={"title": title}).execute()
        doc_id = doc["documentId"]

        # Build full text and formatting requests from markdown.
        full_text, requests = build_requests_and_text(md_content)

        # First, insert the complete text.
        insert_request = {
            "requests": [
                {"insertText": {"location": {"index": 1}, "text": full_text}}
            ]
        }
        service.documents().batchUpdate(documentId=doc_id, body=insert_request).execute()

        # Then, apply formatting requests.
        if requests:
            service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()

        return f"https://docs.google.com/document/d/{doc_id}"
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

def main():
    service = authenticate_docs_service()

    # Markdown meeting notes.
    md_meeting_notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""
    doc_link = create_google_doc(service, "Product Team Sync", md_meeting_notes)
    if doc_link:
        print("Google Doc Created:", doc_link)
    else:
        print("Failed to create document.")

if __name__ == "__main__":
    main()


Google Doc Created: https://docs.google.com/document/d/1lPxcsCRSqwaHOK94vUULDTqD9Exf8mp6CkxK1UwIILE
